# TitaniumCloud feeds
This notebook demonstrates how to use several TitaniumCloud feed API-s (feeds). Those API-s are called feeds due to providing continuous fresh data that can be fetched by applying certain filters and time periods.
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Covered API classes
This notebook covers examples for the following API classes:
- **NewMalwareFilesFeed** (*TCF-0101 New Malware - Files*)
- **MWPChangeEventsFeed** (*TCF-0111 MWP Change Events Feed*)
- **NewMalwarePlatformFiltered** (*TCF-0102-0106 New Malware - Platform-Filtered*)
    - This feed covers product numbers **from TCF-0102 to TCF-0106** and requires separate access rights for each of them.


### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

### 1. Importing the required classes
First, we will import the required API class from the ticloud module.

In [ ]:
from ReversingLabs.SDK.ticloud import NewMalwareFilesFeed, MWPChangeEventsFeed, NewMalwarePlatformFiltered

### 2. Loading the credentials
Next, we will load our TitaniumCloud credentials from the local `ticloud_credentials.json` file.  
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**

In [ ]:
import json


CREDENTIALS = json.load(open("ticloud_credentials.json"))
USERNAME = CREDENTIALS.get("username")
PASSWORD = CREDENTIALS.get("password")
USER_AGENT = json.load(open('../user_agent.json'))["user_agent"]

### 3. New Malware - Files Feed
This feed provides a continuous list of new samples with at least one antivirus scanner detection present in the TitaniumCloud system.  
After we create an object from the NewMalwareFilesFeed class, we will be able to use its methods.

In [ ]:
new_malware_files = NewMalwareFilesFeed(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

#### Pulling with a defined timestamp
To pull the samples with a defined timestamp in the request we can do the following action. This will return the samples from the defined timestamp onwards. The time value can be defined as a Unix timestamp or as UTC time. Here we will demonstrate the use of a Unix timestamp.  
**NOTE:** Make sure to change this timestamp to a preferred date instead before running this code cell.

In [ ]:
response = new_malware_files.pull_with_timestamp(
    time_format="timestamp",
    time_value="1697459211"
)

print(response.text)

#### Setting the start time
Instead of defining the timestamp in our pull request, we can first define the start time for fetching samples and then continunously pull them using a separate method.

In [ ]:
_ = new_malware_files.set_start(
    time_format="timestamp",
    time_value="1697459211"
)

response = new_malware_files.pull()

print(response.text)

### 4. MWP Change Events Feed
The MWP (Malware Presence) Change Events Feed provides a continuous list of new malicious samples, false positives and malicious samples for which the ReversingLabs threat name changed.
After we create an object from the MWPChangeEventsFeed class, we will be able to use its methods.

Inside the ReversingLabs SDK, the MWP Change Events Feed is used precisely the same way as the New Malware - Files Feed:
- You create the feed object from the `MWPChangeEventsFeed` class.
- You can perform a query that carries a timestamp each time:
    - `pull_with_timestamp()
- You can perform a `start()` query to set a starting timestamp and then perform a `pull()` query to start fetching the samples.

Please refer to section 3. for more detail on how to use such queries.

### 5. New Malware - Platform Filtered
This feed returns new detected malware filtered for each selected platform (Android, Boot, ByteCode, Document, DOS, Linux, Win32, ...).  
In this case the feed has a whole range of product names (TCF-0102 to TCF-0106) because usage rights are acquired separately for the mentioned platforms.

#### Feed query
If we use the feed query method, we will be making a fetch request for new malware and also providing a start timestamp for this action with each request.  
The platform filtering is optional but affects the results that we get.  
In our example we will be fetching the results for the Android, Linux and Win32 platforms. Defining the starting timestamp is done the same way as in other feeds in this notebook.  
**NOTE:** Make sure to change the timestamp to a preferred date instead before running this code cell.

In [ ]:
platform_filtered_feed = NewMalwarePlatformFiltered(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

results = platform_filtered_feed.feed_query(
    time_format="timestamp",
    time_value="1697459211",
    platforms=["Android", "Linux", "Win32"]
)

print(results.text)

#### Setting the start time
Just like in the New Malware - Files Feed, we can also set a starting timestamp and then pull all the filtered malware samples from that time onwards with separate requests.

In [ ]:
_ = platform_filtered_feed.start_query(
    time_format="timestamp",
    time_value="1697459211"
)

results = platform_filtered_feed.pull_query(
    platforms=["Android", "Linux", "Win32"]
)

print(results.text)

A full list of available platforms for filtering the results can be found in the official documentation of the New Malware - Platform Filtered feed.  
Keep in mind that the platforms defined in the `platforms` parameter dictate the product name and querying rights.